<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/master/examples/annotation_import/tiled.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/master/examples/annotation_import/tiled.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Tiled Imagery Annotation Import
* This notebook will provide examples of each supported annotation type for tiled imagery assets, and also cover MAL and Label Import methods:

Supported annotations that can be uploaded through the SDK: 
  * Point 
  * Polygon
  * Bounding Box 
  * Classification radio 
  * Classification checklist 
  * Classification free-form text

**Not** supported:
  * Segmentation mask


MAL and Label Import: 

* Model-assisted labeling - used to provide pre-annotated data for your labelers. This will enable a reduction in the total amount of time to properly label your assets. Model-assisted labeling does not submit the labels automatically, and will need to be reviewed by a labeler for submission.
* Label Import - used to provide ground truth labels. These can in turn be used and compared against prediction labels, or used as benchmarks to see how your labelers are doing.

For information on what types of annotations are supported per data type, refer to this documentation:
    * https://docs.labelbox.com/docs/model-assisted-labeling#option-1-import-via-python-annotation-types-recommended

Notes:
 * This notebook uses the Slippy Maps format
 * If you are importing more than 1,000 annotations at a time, consider submitting separate jobs, as they can take longer than other annotation types to import.
 * Wait until the import job is complete before opening the Editor to make sure all annotations are imported properly.
 * You may need to refresh your browser in order to see the results of the import job.

### Setup

In [ ]:
!pip install -q 'labelbox[data]'

In [ ]:
import os
import uuid
import numpy as np
from PIL import Image
import cv2
import ndjson

import labelbox as lb
import labelbox.data.annotation_types as lb_types
import labelbox.schema.queue_mode as lb_queue_mode

from labelbox.data.annotation_types.data.tiled_image import TiledBounds, TiledImageData, TileLayer, EPSG, EPSGTransformer



### Replace with your API key
Guides on [Create an API key](https://docs.labelbox.com/docs/create-an-api-key)

In [ ]:
API_KEY = None
client = lb.Client(API_KEY)

## Supported annotations for tiled imagery

### Supported Python annotation types and NDJSON 

In [ ]:
####### Point #######

# Python Annotation
point_annotation = lb_types.ObjectAnnotation(
  name = "point_geo",
  value = lb_types.Point(x=-122.31741025134123, y=37.87355669249922),
)

# NDJSON
point_annotation_ndjson = {
    "name": "point_geo",
    "point": {
         "x": -122.31741025134123,
         "y": 37.87355669249922
     }
}

In [ ]:
####### Polyline #######
# Coordinates
coords = [ 
            [
              -122.31757789012927,
              37.87396317833991
            ],
            [
              -122.31639782443663,
              37.87396741226917
            ],
            [
              -122.31638977853417,
              37.87277872707839
            ]
        ]

line_points = []
line_points_ndjson = []

for sub in coords: 
  line_points.append(lb_types.Point(x=sub[0], y=sub[1]))
  line_points_ndjson.append({"x":sub[0], "y":sub[1]})

# Python Annotation 
polyline_annotation = lb_types.ObjectAnnotation(
  name = "polyline_geo",
  value = lb_types.Line(points=line_points),
)

# NDJSON 
polyline_annotation_ndjson = {
    "name": "polyline_geo",
    "line": line_points_ndjson
}

In [ ]:
####### Polygon #######
# Coordinates in the desired EPSG coordinate system
coords_polygon = [
    [
        -122.31691812612837,
        37.873289980495024
    ],
    [
        -122.31710184090099,
        37.87304335144298
    ],
    [
        -122.31680146054286,
        37.87303594197371
    ],
    [
        -122.31691812612837,
        37.873289980495024
    ]
]

polygon_points = []
polygon_points_ndjson = []

for sub in coords_polygon: 
  polygon_points.append(lb_types.Point(x=sub[0], y=sub[1]))
  polygon_points_ndjson.append({"x":sub[0], "y":sub[1]})

# Python Annotation 
polygon_annotation = lb_types.ObjectAnnotation(
  name = "polygon_geo",
  value = lb_types.Polygon(points=polygon_points),
)

# NDJSON 
polygon_annotation_ndjson = {
    "name": "polygon_geo",
    "polygon": polygon_points_ndjson
}

In [ ]:
####### Bounding Box #######
coord_object = {
      "coordinates": [
        [
          [
              -122.31734455895823,
              37.873713376083884
          ],
          [
              -122.31734455895823,
              37.87385944699745
          ],
          [
              -122.31673038840458,
              37.87385944699745
          ],
          [
              -122.31673038840458,
              37.873713376083884
          ],
          [
              -122.31734455895823,
              37.873713376083884
          ]
        ]
      ]  
    }


bbox_top_left = lb_types.Point(x=-122.31734455895823, y=37.873713376083884)
bbox_bottom_right = lb_types.Point(x=-122.31673038840458, y=37.87385944699745)

# Python Annotation
bbox_annotation = lb_types.ObjectAnnotation(
  name = "bbox_geo",
  value = lb_types.Rectangle(start=bbox_top_left, end=bbox_bottom_right)
)


# NDJSON
bbox_annotation_ndjson = {
    "name" : "bbox_geo",
    "bbox" : {
        'top': coord_object["coordinates"][0][1][1],
        'left': coord_object["coordinates"][0][1][0],
        'height': coord_object["coordinates"][0][3][1] - coord_object["coordinates"][0][1][1],        
        'width': coord_object["coordinates"][0][3][0] - coord_object["coordinates"][0][1][0]
    }
}


In [ ]:
####### Classification - radio (single choice) #######

# Python Annotation 
radio_annotation = lb_types.ClassificationAnnotation(
    name="radio_question_geo", 
    value=lb_types.Radio(answer=lb_types.ClassificationAnswer(name="first_radio_answer"))
)


# NDJSON 
radio_annotation_ndjson = {
    "name": "radio_question_geo",
    "answer": { "name": "first_radio_answer"}
}

In [ ]:
####### Classification - Checklist (multi-choice) #######

coord_object_checklist = {
    "coordinates": [
       [
          [
              -122.31711256877092,
              37.87340218056304
          ],
          [
              -122.31711256877092,
              37.87360752741479
          ],
          [
              -122.31665529331502,
              37.87360752741479
          ],
          [
              -122.31665529331502,
              37.87340218056304
          ],
          [
              -122.31711256877092,
              37.87340218056304
          ]
      ]
    ]          
}

# Python Annotation
bbox_with_checklist_subclass = lb_types.ObjectAnnotation(
    name="bbox_checklist_geo",
    value=lb_types.Rectangle(
        start=lb_types.Point(x=-122.31711256877092, y=37.87340218056304), # Top left
        end=lb_types.Point(x=-122.31665529331502, y=37.87360752741479), # Bottom right
    ),
    classifications=[
        lb_types.ClassificationAnnotation(
            name="checklist_class_name",
            value=lb_types.Checklist(
                answer=[lb_types.ClassificationAnswer(name="first_checklist_answer")]
            )
        )
    ]
)


# NDJSON 
bbox_with_checklist_subclass_ndjson = {
    "name": "bbox_checklist_geo", 
    "classifications": [{
        "name": "checklist_class_name",
        "answer": [
            { "name":"first_checklist_answer" }
        ]   
    }],
    "bbox": {
        'top': coord_object_checklist["coordinates"][0][1][1],
        'left': coord_object_checklist["coordinates"][0][1][0],
        'height': coord_object_checklist["coordinates"][0][3][1] - coord_object_checklist["coordinates"][0][1][1],        
        'width': coord_object_checklist["coordinates"][0][3][0] - coord_object_checklist["coordinates"][0][1][0]
    }
}

In [ ]:
####### Classification free form text with bbox #######

coord_object_text ={
    "coordinates": [
      [
          [
              -122.31750814315438,
              37.87318201423049
          ],
          [
              -122.31750814315438,
              37.87337992476082
          ],
          [
              -122.31710049991725,
              37.87337992476082
          ],
          [
              -122.31710049991725,
              37.87318201423049
          ],
          [
              -122.31750814315438,
              37.87318201423049
          ]
      ]
    ]
}
# Python Annotation
bbox_with_free_text_subclass = lb_types.ObjectAnnotation(
    name="bbox_text_geo",
    value=lb_types.Rectangle(
        start=lb_types.Point(x=-122.31750814315438, y=37.87318201423049), # Top left
        end=lb_types.Point(x=-122.31710049991725, y=37.87337992476082), # Bottom right
    ),
    classifications=[
        lb_types.ClassificationAnnotation(
            name="free_text_geo",
            value=lb_types.Text(answer="sample text")
        )
    ]
)

# NDJSON 
bbox_with_free_text_subclass_ndjson = {
    "name":"bbox_text_geo",
    "classifications": [{
        "name": "free_text_geo",
        "answer": "sample text"
    }],
    "bbox": {
        'top': coord_object_text["coordinates"][0][1][1],
        'left': coord_object_text["coordinates"][0][1][0],
        'height': coord_object_text["coordinates"][0][3][1] - coord_object_text["coordinates"][0][1][1],        
        'width': coord_object_text["coordinates"][0][3][0] - coord_object_text["coordinates"][0][1][0]
    }
}

In [ ]:
####### Classification - Checklist (multi-choice) #######

# Python Annotation
checklist_annotation = lb_types.ClassificationAnnotation(
    name="checklist_question_geo",
    value=lb_types.Checklist(answer = [
        lb_types.ClassificationAnswer(name = "first_checklist_answer"),
        lb_types.ClassificationAnswer(name = "second_checklist_answer"),
        lb_types.ClassificationAnswer(name = "third_checklist_answer")
    ])
  )



# NDJSON
checklist_annotation_ndjson = {
  'name': 'checklist_question_geo',
  'answer': [
    {'name': 'first_checklist_answer'},
    {'name': 'second_checklist_answer'},
    {'name': 'third_checklist_answer'},
  ]
}

In [ ]:
###### Nested Classifications  ########
nested_radio_classification_ndjson = {
  'name': 'radio_question_nested',
  'answer': {'name': 'first_radio_question'},
  'classifications' : [
    {'name': 'sub_question_radio', 'answer': {'name': 'sub_answer'}}
   ]
}

nested_checklist_annotation_ndjson = {
  "name": "nested_checklist_question",
  "answer": [{
      "name": "first_checklist_answer",
      "classifications" : [
        {
          "name": "sub_checklist_question",
          "answer": {"name": "first_sub_checklist_answer"}
        }
      ]
  }]
}

## Upload Annotations - putting it all together


### Step 1: Import data rows into Catalog

In [ ]:

top_left_bound = lb_types.Point(x=-122.31764674186705, y=37.87276155898985)
bottom_right_bound = lb_types.Point(x=-122.31635199317932, y=37.87398109727749)

epsg = EPSG.EPSG4326
bounds = TiledBounds(epsg=epsg, bounds=[top_left_bound, bottom_right_bound])

tile_layer = TileLayer(
    url="https://api.mapbox.com/styles/v1/mapbox/satellite-streets-v11/tiles/{z}/{x}/{y}?access_token=pk.eyJ1IjoibWFwYm94IiwiYSI6ImNpejY4NXVycTA2emYycXBndHRqcmZ3N3gifQ.rJcFIG214AriISLbB6B5aw"
)

tiled_image_data = TiledImageData(tile_layer=tile_layer,
                                  tile_bounds=bounds,
                                  zoom_levels=[17, 23])

dataset = client.create_dataset(name="geospatial_demo_dataset")
task = dataset.create_data_rows([tiled_image_data.asdict()])
task.wait_till_done()
print(task.errors)

None


### Step 2: Create/select an ontology
Your project should have the correct ontology setup with all the tools and classifications supported for your annotations, and the tool names and classification instructions should match the `name`/`instructions` fields in your annotations to ensure the correct feature schemas are matched.

In [ ]:
ontology_builder = lb.OntologyBuilder(
    tools=[
        lb.Tool(tool=lb.Tool.Type.POINT, name="point_geo"),
        lb.Tool(tool=lb.Tool.Type.LINE, name="polyline_geo"),
        lb.Tool(tool=lb.Tool.Type.POLYGON, name="polygon_geo"),
        lb.Tool(tool=lb.Tool.Type.POLYGON, name="polygon_geo_2"),
        lb.Tool(tool=lb.Tool.Type.BBOX, name="bbox_geo"), 
        lb.Tool( 
          tool=lb.Tool.Type.BBOX, 
          name="bbox_checklist_geo",
          classifications=[
                lb.Classification(
                    class_type=lb.Classification.Type.CHECKLIST,
                    name="checklist_class_name",
                    options=[
                      lb.Option(value="first_checklist_answer")
                    ]
                ),
            ]
          ),
        lb.Tool( 
          tool=lb.Tool.Type.BBOX, 
          name="bbox_text_geo",
          classifications=[
                lb.Classification(
                    class_type=lb.Classification.Type.TEXT,
                    name="free_text_geo"
                ),
            ]
          )    
      ],
      classifications = [
          lb.Classification(
              class_type=lb.Classification.Type.CHECKLIST, 
              name="checklist_question_geo",
              options=[
                  lb.Option(value="first_checklist_answer"),
                  lb.Option(value="second_checklist_answer"), 
                  lb.Option(value="third_checklist_answer")
              ]
          ), 
          lb.Classification(
              class_type=lb.Classification.Type.RADIO, 
              name="radio_question_geo",
              options=[
                  lb.Option(value="first_radio_answer")
              ]
          ),
          
         lb.Classification(
              class_type=lb.Classification.Type.RADIO,
              name="radio_question_nested",
              options=[
                  lb.Option("first_radio_question",
                        options=[
                            lb.Classification(
                                class_type=lb.Classification.Type.RADIO,
                                name="sub_question_radio",
                                options=[lb.Option("sub_answer")]
                            )
                        ]
                  )
              ] 
        ),
        lb.Classification(
          class_type=lb.Classification.Type.CHECKLIST,
          name="nested_checklist_question",
          options=[
              lb.Option("first_checklist_answer",
                options=[
                  lb.Classification(
                      class_type=lb.Classification.Type.CHECKLIST,
                      name="sub_checklist_question",
                      options=[lb.Option("first_sub_checklist_answer")]
                  )
              ]
            )
          ]
        )
          
      ]
)

ontology = client.create_ontology("Ontology Geospatial Annotations", ontology_builder.asdict(), media_type=lb.MediaType.Geospatial_Tile)

### Step 3: Create a labeling project
Connect the ontology to the labeling project 

In [ ]:
# Project defaults to batch mode with benchmark quality settings if this argument is not provided
# Queue mode will be deprecated once dataset mode is deprecated

project = client.create_project(name="geospatial_project_demo",
                                    queue_mode=lb_queue_mode.QueueMode.Batch,
                                    media_type=lb.MediaType.Geospatial_Tile)


project.setup_editor(ontology)

######################### DATASET CONSENSUS OPTION ########################
#Note that dataset base projects will be deprecated in the near future.

#To use Datasets/Consensus instead of Batches/Benchmarks use the following query: 
#In this case, 10% of all data rows need to be annotated by three labelers.

# dataset_project = client.create_project(name="dataset-test-project",
#                                 description="a description",
#                                 media_type=MediaType.Text,
#                                 auto_audit_percentage=0.1,
#                                 auto_audit_number_of_labels=3,
#                                 queue_mode=QueueMode.Dataset)

# dataset_project.datasets.connect(dataset)

### Step 4: Send a batch of data rows to the project 

In [ ]:
# Setup Batches and Ontology

# Create a batch to send to your MAL project
batch = project.create_batch(
  "first-batch-geo-demo", # Each batch in a project must have a unique name
  dataset.export_data_rows(), # Paginated collection of data row objects
  5 # priority between 1(Highest) - 5(lowest)
)

print("Batch: ", batch)

Batch:  <Batch {
    "consensus_settings_json": "{\"numberOfLabels\":1,\"coveragePercentage\":0}",
    "created_at": "2023-02-09 21:29:42+00:00",
    "name": "first-batch-geo-demo",
    "size": 1,
    "uid": "dd7198b0-a8c0-11ed-bc96-d38ba1ad90ac",
    "updated_at": "2023-02-09 21:29:42+00:00"
}>


### Step 5: Create the annotations payload 
Create the annotations payload using the snippets of code above

Labelbox support two formats for the annotations payload: NDJSON and Python Annotation types. Both are described below. 


#### Python annotations
Here we create the complete label ndjson payload of annotations only using python annotation format. There is one annotation for each reference to an annotation that we created on ***Supported Python annotation types and NDJSON*** section.

In [ ]:
 ## Lets create another polygon annotation with python annotation tools that draws the image using cv2 and PIL python libraries

hsv = cv2.cvtColor(tiled_image_data.value, cv2.COLOR_RGB2HSV)
mask = cv2.inRange(hsv, (10, 25, 25), (100, 150, 255))
kernel = np.ones((15, 20), np.uint8)
mask = cv2.erode(mask, kernel)
mask = cv2.dilate(mask, kernel)
mask_annotation = lb_types.MaskData.from_2D_arr(mask)
mask_data = lb_types.Mask(mask=mask_annotation, color=[255, 255, 255])
h, w, _ = tiled_image_data.value.shape
pixel_bounds = TiledBounds(epsg=EPSG.SIMPLEPIXEL,
                          bounds=[lb_types.Point(x=0, y=0),
                                  lb_types.Point(x=w, y=h)])
transformer = EPSGTransformer.create_pixel_to_geo_transformer(
    src_epsg=pixel_bounds.epsg,
    pixel_bounds=pixel_bounds,
    geo_bounds=tiled_image_data.tile_bounds,
    zoom=23)
pixel_polygons = mask_data.shapely.simplify(3)
list_of_polygons = [transformer(lb_types.Polygon.from_shapely(p)) for p in pixel_polygons.geoms]
polygon_annotation_two = lb_types.ObjectAnnotation(value=list_of_polygons[0], name="polygon_geo_2")


# Here is the NDJSON representation of the resulting polygon:
polygon_annotation_two_ndjson = {
    "name": "polygon_geo_2",
    "polygon": [
        {'x': -122.31703039689702, 'y': 37.87397804081582},
        {'x': -122.31702351036107, 'y': 37.87393525033866},
        {'x': -122.31698907768116, 'y': 37.87389857276706},
        {'x': -122.3169787478772, 'y': 37.87385883871054},
        {'x': -122.31695808826926, 'y': 37.87385578224377},
        {'x': -122.31695464500127, 'y': 37.873816048164166},
        {'x': -122.31692021232138, 'y': 37.873779370533214},
        {'x': -122.31690988251741, 'y': 37.87373352346883},
        {'x': -122.3168857796415, 'y': 37.873696845796786},
        {'x': -122.3168547902296, 'y': 37.873684619902065},
        {'x': -122.31682035754969, 'y': 37.873611264491025},
        {'x': -122.31676526526188, 'y': 37.87355013492598},
        {'x': -122.3167583787259, 'y': 37.87351651364362},
        {'x': -122.31671017297403, 'y': 37.87348900531027},
        {'x': -122.31671017297403, 'y': 37.873452327516496},
        {'x': -122.31667918356217, 'y': 37.87344010158117},
        {'x': -122.31663442107829, 'y': 37.87335451997715},
        {'x': -122.31660343166638, 'y': 37.87334840700161},
        {'x': -122.31659998839841, 'y': 37.873320898605485},
        {'x': -122.31654489611057, 'y': 37.87329033370888},
        {'x': -122.31652767977064, 'y': 37.87319863894286},
        {'x': -122.31648980382273, 'y': 37.8731833564708},
        {'x': -122.31648980382273, 'y': 37.873161961004534},
        {'x': -122.31641749519497, 'y': 37.87309166157168},
        {'x': -122.316410608659, 'y': 37.873054983580076},
        {'x': -122.31639683558704, 'y': 37.873039701078184},
        {'x': -122.31635551637117, 'y': 37.873039701078184},
        {'x': -122.31635551637117, 'y': 37.87398109727749},
        {'x': -122.31703039689702, 'y': 37.87397804081582}
    ]
}

In [ ]:

tiled_image_data_row_id = next(dataset.export_data_rows()).uid

label = lb_types.Label(
    data=TiledImageData(
        uid=tiled_image_data_row_id ,
        tile_layer=tile_layer,
        tile_bounds=bounds,
        zoom_levels=[17, 23]
    ),
    annotations = [
        point_annotation,
        polyline_annotation,
        polygon_annotation,
        bbox_annotation,
        radio_annotation,
        bbox_with_checklist_subclass,  
        bbox_with_free_text_subclass,
        checklist_annotation,
        polygon_annotation_two
    ]
)

[{'uuid': '61ad497c-85bd-4f3c-a32e-4cb368844a22',
  'dataRow': {'id': 'cldxm4s5906rw074nfm5kgtrl'},
  'name': 'point_geo',
  'classifications': [],
  'point': {'x': -122.31741025134123, 'y': 37.87355669249922}},
 {'uuid': '5d6f057f-a0cd-4e9f-b9c2-ade883874226',
  'dataRow': {'id': 'cldxm4s5906rw074nfm5kgtrl'},
  'name': 'polyline_geo',
  'classifications': [],
  'line': [{'x': -122.31757789012927, 'y': 37.87396317833991},
   {'x': -122.31639782443663, 'y': 37.87396741226917},
   {'x': -122.31638977853417, 'y': 37.87277872707839}]},
 {'uuid': '7f2190d0-ba76-4c26-9c95-5b152f1e522d',
  'dataRow': {'id': 'cldxm4s5906rw074nfm5kgtrl'},
  'name': 'polygon_geo',
  'classifications': [],
  'polygon': [{'x': -122.31691812612837, 'y': 37.873289980495024},
   {'x': -122.31710184090099, 'y': 37.87304335144298},
   {'x': -122.31680146054286, 'y': 37.87303594197371},
   {'x': -122.31691812612837, 'y': 37.873289980495024}]},
 {'uuid': '2c7b71f2-b0dc-4ea6-85bf-363794c1dac7',
  'dataRow': {'id': 'cldxm4

### NDJSON annotations
Here we create the complete label NDJSON payload of annotations only using NDJSON format. There is one annotation for each reference to an annotation that we created on *** Supported Python annotation types and NDJSON *** section.

In [ ]:
label_ndjson_method2 = []

for annotations in [point_annotation_ndjson,
                    polyline_annotation_ndjson,
                    polygon_annotation_ndjson,
                    bbox_annotation_ndjson,
                    radio_annotation_ndjson,
                    bbox_with_checklist_subclass_ndjson,  
                    bbox_with_free_text_subclass_ndjson,
                    checklist_annotation_ndjson,
                    nested_checklist_annotation_ndjson,
                    nested_radio_classification_ndjson,
                    polygon_annotation_two_ndjson
                    ]:
  annotations.update({
      'uuid' : str(uuid.uuid4()),
      'dataRow': {
          'id':  next(dataset.export_data_rows()).uid
      }
  })
  label_ndjson_method2.append(annotations)
  

In [ ]:
label_ndjson_method2

[{'name': 'point_geo',
  'point': {'x': -122.31741025134123, 'y': 37.87355669249922},
  'uuid': 'b148343c-031b-40d5-a22c-6fe950f6c787',
  'dataRow': {'id': 'cldxm4s5906rw074nfm5kgtrl'}},
 {'name': 'polyline_geo',
  'line': [{'x': -122.31757789012927, 'y': 37.87396317833991},
   {'x': -122.31639782443663, 'y': 37.87396741226917},
   {'x': -122.31638977853417, 'y': 37.87277872707839}],
  'uuid': 'd087ec4e-80f7-4a96-8f32-77102883dca8',
  'dataRow': {'id': 'cldxm4s5906rw074nfm5kgtrl'}},
 {'name': 'polygon_geo',
  'polygon': [{'x': -122.31691812612837, 'y': 37.873289980495024},
   {'x': -122.31710184090099, 'y': 37.87304335144298},
   {'x': -122.31680146054286, 'y': 37.87303594197371},
   {'x': -122.31691812612837, 'y': 37.873289980495024}],
  'uuid': '2bbd457a-dd3b-4647-b159-2222e2565cb1',
  'dataRow': {'id': 'cldxm4s5906rw074nfm5kgtrl'}},
 {'name': 'bbox_geo',
  'bbox': {'top': 37.87385944699745,
   'left': -122.31734455895823,
   'height': -0.0001460709135656657,
   'width': 0.0006141705

### Step 6: Upload annotations to a project as pre-labels or complete labels


#### Model-Assisted Labeling (MAL)
For the purpose of this tutorial only run one of the label_ndjosn  annotation type tools at the time (NDJSON or Annotation types). Delete the previous labels before uploading labels that use the 2nd method (ndjson)

In [ ]:
# Upload MAL label for this data row in project
upload_job = lb.MALPredictionImport.create_from_objects(
    client = client, 
    project_id = project.uid, 
    name="mal_import_job"+str(uuid.uuid4()), 
    ### use label_ndjson_method2 if labels were created using NDJSON tools
    predictions=label_ndjson_method2)

upload_job.wait_until_done();
print("Errors:", upload_job.errors)
print("   ")

Errors: []
   


#### Label Import

In [ ]:
# Upload label for this data row in project 
upload_job = lb.LabelImport.create_from_objects(
    client = client, 
    project_id = project.uid, 
    name="label_geo_import_job"+str(uuid.uuid4()),  
    # user label_ndjson if labels were created using python annotation tools
    labels=label_ndjson_method2)

upload_job.wait_until_done();
print("Errors:", upload_job.errors)

Errors: []


### Optional deletions for cleanup

In [ ]:

# project.delete()
# dataset.delete()